In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
 !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip /content/train.zip
print('done')

In [ ]:
%mkdir images

In [ ]:
%mv train images

In [ ]:
class CatDogDataset(torch.utils.data.Dataset):                             #data generating class
    def __init__(self, file_list, dir, mode='train', transform = None):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
        if self.mode == 'train':
            if 'dog' in self.file_list[0]:
                self.label = 1
            else:
                self.label = 0
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dir, self.file_list[idx]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]

transform1 = transform.Compose([
    transform.RandomResizedCrop(224),  
    transform.ToTensor(),
    transform.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [ ]:
train_dir = 'images/train'
cat_files = []
dog_files = []

cat_files= [i for i in os.listdir('images/train') if 'cat' in i]
dog_files = [i for i in os.listdir('images/train') if 'dog' in i]

cats = CatDogDataset(cat_files, train_dir, transform = transform1)
dogs = CatDogDataset(dog_files, train_dir, transform = transform1)


In [ ]:
catdogs = torch.utils.data.ConcatDataset([cats,dogs])

In [ ]:
catdogs
dataloader = torch.utils.data.DataLoader(catdogs, batch_size = 32, shuffle=True)

In [ ]:
images, labels = iter(dataloader).next()

In [ ]:
classes = {}
classes[1] = 'dog'
classes[0] = 'cat'

In [ ]:
npimg = images[4].numpy()
npimg = np.transpose(npimg,(1,2,0))

std_corr = np.asarray([0.229, 0.224, 0.225])
mean = np.asarray([0.485, 0.456, 0.406])

npimg = np.multiply(npimg,std_corr) + mean
plt.imshow(npimg)
print(classes[labels[4].item()])
print(images[1].shape)

Importing libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
import torchvision.datasets as datasets

import warnings
warnings.filterwarnings('ignore')
from PIL import Image

In [ ]:
def imshow(images,title):
  npimg = images.numpy()
  
  std_corr = np.asarray((0.229, 0.224, 0.225)).reshape(3,1,1)
  mean = np.asarray((0.485, 0.456, 0.406)).reshape(3,1,1)
  
  npimg = np.multiply(npimg , std_corr) + mean
  npimg = np.transpose(npimg, (1,2,0))

  plt.figure(figsize=(batchsize*4,4))
  plt.imshow(npimg)
  plt.title(title)
  plt.show()

In [ ]:
batchsize=4
def show_batch():
  dataloader = torch.utils.data.DataLoader(catdogs,batch_size=batchsize,shuffle=True)
  dataiter = iter(dataloader)
  
  images,labels = next(dataiter)

  output = model.forward(images)
  a, pred = torch.max(output.data,1)
  print(a)
  imshow(torchvision.utils.make_grid(images),[classes[x.item()] for x in pred])


In [ ]:
rest = torchvision.models.alexnet(pretrained=True)

In [ ]:
rest

In [ ]:
in_features = rest.classifier[6].in_features
rest.classifier  = nn.Sequential(
    nn.Dropout(p=0.5,inplace=False),
    nn.Linear(in_features=9216, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4096, out_features=2, bias=True),
    nn.Softmax(dim=1)
)



In [ ]:
rest

In [ ]:
for params in rest.parameters():
  if params.requires_grad:
    print(params.shape)

In [ ]:
batch_size = 1
def evaluation(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        outputs = nn.functional.softmax(outputs, dim=1)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total


In [ ]:
import copy
device = torch.device('cuda:0')
loss_epc=[]
def fit(model,trainloader,opt,loss_fn,epochs):
  
  min_loss = 1000
  for j in tqdm_notebook(range(epochs),total=epochs ,unit= 'epoch'): 
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = model.forward(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
      
        
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
        if i%200 == 0:
          print('For epoch [{}/{}]: Training error after {} iterations is {}'.format(j,epochs,i,loss.item()))
          
          if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(vgg.state_dict())
            print('Min loss %0.2f' % min_loss)

    print('Loss after {} epochs is {}'.format(j,loss.item()))    
    loss_epc.append(loss.item())
        
    print('The training accuracy is, {}'.format(evaluation(trainloader,model)))
  
  plt.plot(loss_epc,'red')
  plt.xlabel('epochs')
  plt.ylabel('error')

In [ ]:
rest = rest.to(device)
loss_fn  = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rest.parameters(),lr=0.001)

In [ ]:
from tqdm.notebook import tqdm_notebook 
fit(rest,dataloader,optimizer,loss_fn,10)

In [ ]:
!unzip /content/test1.zip

In [ ]:
%mkdir images_test
%mv test1 images_test

In [ ]:
filename_pth = 'ckpt_densenet121_catdog.pth'
torch.save(rest.state_dict(), filename_pth)

In [ ]:
test_dir = 'images_test/test1'
test_files = os.listdir(test_dir)
test_transform = transform.Compose([
    transform.RandomResizedCrop(224),
    transform.ToTensor()
])

In [ ]:
testset = CatDogDataset(test_files,test_dir,mode ='test',transform=test_transform)

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=32,shuffle=False)

In [ ]:
rest.eval()
fn_list,pred_list = [], []
for x,fn in test_loader:
  with torch.no_grad():
    x = x.to(device)
    out = rest.forward(x)
    pred = torch.argmax(out.data,1)
    fn_list += [n[:-4] for n in fn]
    pred_list += [p.item() for p in pred]

In [ ]:
import pandas as pd
submission = pd.DataFrame({'id':fn_list,'label':pred_list})


In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
/cat
